<a href="https://colab.research.google.com/github/Altaieb-Mohammed/lab_2corse/blob/master/c_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:



def calculate_inheritance(estate, heirs):
    """
    خوارزمية توزيع الميراث حسب نصوص القرآن الكريم (نسخة مبسطة)
    estate: قيمة التركة (رقم)
    heirs: قاموس يحتوي على بيانات الورثة وعددهم
    """
    # 1. تحديد نصيب الزوج/الزوجة
    husband_share = 0
    wife_share = 0
    if heirs.get("زوج", False):
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0:
            husband_share = 1/4
        else:
            husband_share = 1/2

    if heirs.get("زوجات", 0) > 0:
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0:
            wife_share = 1/8
        else:
            wife_share = 1/4

    # 2. نصيب الأم
    mother_share = 0
    if heirs.get("أم", False):
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0 or heirs.get("إخوة_ذكور", 0) + heirs.get("إخوة_إناث", 0) > 0:
            mother_share = 1/6
        else:
            mother_share = 1/3

    # 3. نصيب الأب
    father_share = 0
    father_is_asaba = False
    if heirs.get("أب", False):
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0 or heirs.get("إخوة_ذكور", 0) > 0:
            father_share = 1/6
        else:
            # الأب يأخذ 1/6 + باقي التركة (عصبة)
            father_share = 1/6
            father_is_asaba = True

    # 4. نصيب البنات
    daughters_share = 0
    daughters_count = heirs.get("بنات", 0)
    sons_count = heirs.get("أبناء_ذكور", 0)
    if daughters_count > 0:
        if sons_count == 0:
            if daughters_count == 1:
                daughters_share = 1/2
            elif daughters_count >= 2:
                daughters_share = 2/3
        else:
            daughters_share = 0  # يرثن مع الأولاد حسب العصبة

    # 5. حساب مجموع الفروض
    total_fixed_shares = husband_share + wife_share + mother_share + father_share + daughters_share

    # 6. التعامل مع حالة تجاوز مجموع الفروض 1 (الرد)
    if total_fixed_shares > 1:
        # تقليل الأنصبة بنسبة متناسبة
        factor = 1 / total_fixed_shares
        husband_share *= factor
        wife_share *= factor
        mother_share *= factor
        father_share *= factor
        daughters_share *= factor
        total_fixed_shares = 1

    # 7. توزيع الباقي (العصبة)
    remainder = 1 - total_fixed_shares

    # تحديد العصبة (الأبناء الذكور أولاً، ثم الأب إذا هو عصبة)
    asaba = {}
    total_asaba_units = 0

    if sons_count > 0:
        # كل ذكر يعادل وحدتين (لأن الذكر يأخذ ضعف الأنثى)
        asaba["أبناء_ذكور"] = sons_count * 2
        total_asaba_units += sons_count * 2

        if daughters_count > 0:
            asaba["بنات"] = daughters_count * 1
            total_asaba_units += daughters_count * 1

    else:
        # لا أبناء ذكور
        if father_is_asaba:
            asaba["أب"] = 1
            total_asaba_units += 1

    # 8. حساب نصيب العصبة من الباقي
    shares = {}

    # توزيع الفروض
    if husband_share > 0:
        shares["زوج"] = husband_share * estate
    if wife_share > 0:
        shares["زوجة"] = wife_share * estate
    if mother_share > 0:
        shares["أم"] = mother_share * estate
    if father_share > 0:
        shares["أب"] = father_share * estate
    if daughters_share > 0 and sons_count == 0:
        # البنات يرثن كنصيب فرض فقط
        shares["بنات"] = daughters_share * estate

    # توزيع العصبة
    if remainder > 0 and total_asaba_units > 0:
        unit_value = remainder * estate / total_asaba_units
        for key, units in asaba.items():
            shares[key] = shares.get(key, 0) + units * unit_value

    # 9. طباعة النتائج
    print(f"\nقيمة التركة: {estate}")
    print("توزيع الميراث:")
    for heir, amount in shares.items():
        print(f"{heir}: {amount:.2f} ({amount/estate:.3%} من التركة)")

    return shares


# مجموعة حالات الاختبار
test_cases = [
    {
        "description": "حالة 1: زوج، أب، أم، 2 أبناء ذكور، 1 بنت",
        "estate": 100000,
        "heirs": {
            "زوج": True,
            "زوجات": 0,
            "أب": True,
            "أم": True,
            "أبناء_ذكور": 2,
            "بنات": 1,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 2: زوجة، أم، 3 بنات، لا أبناء ذكور",
        "estate": 120000,
        "heirs": {
            "زوج": False,
            "زوجات": 1,
            "أب": False,
            "أم": True,
            "أبناء_ذكور": 0,
            "بنات": 3,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 3: أب، أم، بنت واحدة، لا زوج ولا أبناء ذكور",
        "estate": 80000,
        "heirs": {
            "زوج": False,
            "زوجات": 0,
            "أب": True,
            "أم": True,
            "أبناء_ذكور": 0,
            "بنات": 1,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 4: زوج، لا أب، لا أم، 1 ابن ذكر، 2 بنات",
        "estate": 50000,
        "heirs": {
            "زوج": True,
            "زوجات": 0,
            "أب": False,
            "أم": False,
            "أبناء_ذكور": 1,
            "بنات": 2,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 5: أم فقط، لا أب، لا أبناء، لا زوج",
        "estate": 30000,
        "heirs": {
            "زوج": False,
            "زوجات": 0,
            "أب": False,
            "أم": True,
            "أبناء_ذكور": 0,
            "بنات": 0,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    }
]

# تنفيذ الاختبارات
for case in test_cases:
    print("\n" + "="*50)
    print(case["description"])
    calculate_inheritance(case["estate"], case["heirs"])


حالة 1: زوج، أب، أم، 2 أبناء ذكور، 1 بنت

قيمة التركة: 100000
توزيع الميراث:
زوج: 25000.00 (25.000% من التركة)
أم: 16666.67 (16.667% من التركة)
أب: 16666.67 (16.667% من التركة)
أبناء_ذكور: 33333.33 (33.333% من التركة)
بنات: 8333.33 (8.333% من التركة)

حالة 2: زوجة، أم، 3 بنات، لا أبناء ذكور

قيمة التركة: 120000
توزيع الميراث:
زوجة: 15000.00 (12.500% من التركة)
أم: 20000.00 (16.667% من التركة)
بنات: 80000.00 (66.667% من التركة)

حالة 3: أب، أم، بنت واحدة، لا زوج ولا أبناء ذكور

قيمة التركة: 80000
توزيع الميراث:
أم: 13333.33 (16.667% من التركة)
أب: 13333.33 (16.667% من التركة)
بنات: 40000.00 (50.000% من التركة)

حالة 4: زوج، لا أب، لا أم، 1 ابن ذكر، 2 بنات

قيمة التركة: 50000
توزيع الميراث:
زوج: 12500.00 (25.000% من التركة)
أبناء_ذكور: 18750.00 (37.500% من التركة)
بنات: 18750.00 (37.500% من التركة)

حالة 5: أم فقط، لا أب، لا أبناء، لا زوج

قيمة التركة: 30000
توزيع الميراث:
أم: 10000.00 (33.333% من التركة)


In [ ]:
#Gg

def calculate_inheritance(estate, heirs, debts=0, wills=0):
    """
    توزيع الميراث مع خصم الديون والوصايا.
    estate: قيمة التركة الكلية قبل الخصم
    heirs: بيانات الورثة وعددهم (قاموس)
    debts: مجموع الديون المستحقة على المتوفى (رقم)
    wills: قيمة الوصايا (يجب ألا تتجاوز ثلث التركة بعد خصم الديون)
    """

    print(f"التركة الكلية قبل الخصم: {estate}")
    print(f"الديون المستحقة: {debts}")
    print(f"الوصايا (حتى ثلث التركة المتبقية): {wills}")

    # 1. خصم الديون أولاً
    if debts > estate:
        print("الديون أكبر من التركة، لا ميراث للورثة.")
        return {}
    estate_after_debts = estate - debts

    # 2. خصم الوصايا (حتى ثلث التركة بعد خصم الديون)
    max_will = estate_after_debts / 3
    if wills > max_will:
        print(f"الوصايا تتجاوز الثلث المسموح به ({max_will}). سيتم تقليلها إلى الثلث.")
        wills = max_will
    estate_for_inheritance = estate_after_debts - wills

    print(f"التركة المتبقية للتوزيع بعد خصم الديون والوصايا: {estate_for_inheritance}")

    # 3. نصيب الزوج/الزوجة
    husband_share = 0
    wife_share = 0
    if heirs.get("زوج", False):
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0:
            husband_share = 1/4
        else:
            husband_share = 1/2

    if heirs.get("زوجات", 0) > 0:
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0:
            wife_share = 1/8
        else:
            wife_share = 1/4

    # 4. نصيب الأم
    mother_share = 0
    if heirs.get("أم", False):
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0 or heirs.get("إخوة_ذكور", 0) + heirs.get("إخوة_إناث", 0) > 0:
            mother_share = 1/6
        else:
            mother_share = 1/3

    # 5. نصيب الأب
    father_share = 0
    father_is_asaba = False
    if heirs.get("أب", False):
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0 or heirs.get("إخوة_ذكور", 0) > 0:
            father_share = 1/6
        else:
            father_share = 1/6
            father_is_asaba = True

    # 6. نصيب البنات
    daughters_share = 0
    daughters_count = heirs.get("بنات", 0)
    sons_count = heirs.get("أبناء_ذكور", 0)
    if daughters_count > 0:
        if sons_count == 0:
            if daughters_count == 1:
                daughters_share = 1/2
            elif daughters_count >= 2:
                daughters_share = 2/3
        else:
            daughters_share = 0  # يرثن مع الأولاد حسب العصبة

    # 7. مجموع الفروض
    total_fixed_shares = husband_share + wife_share + mother_share + father_share + daughters_share

    # 8. الرد إذا تجاوزت الفروض التركة
    if total_fixed_shares > 1:
        factor = 1 / total_fixed_shares
        husband_share *= factor
        wife_share *= factor
        mother_share *= factor
        father_share *= factor
        daughters_share *= factor
        total_fixed_shares = 1

    # 9. الباقي للعصبة
    remainder = 1 - total_fixed_shares

    asaba = {}
    total_asaba_units = 0

    if sons_count > 0:
        asaba["أبناء_ذكور"] = sons_count * 2
        total_asaba_units += sons_count * 2
        if daughters_count > 0:
            asaba["بنات"] = daughters_count * 1
            total_asaba_units += daughters_count * 1
    else:
        if father_is_asaba:
            asaba["أب"] = 1
            total_asaba_units += 1

    shares = {}

    # توزيع الفروض
    if husband_share > 0:
        shares["زوج"] = husband_share * estate_for_inheritance
    if wife_share > 0:
        shares["زوجة"] = wife_share * estate_for_inheritance
    if mother_share > 0:
        shares["أم"] = mother_share * estate_for_inheritance
    if father_share > 0:
        shares["أب"] = father_share * estate_for_inheritance
    if daughters_share > 0 and sons_count == 0:
        shares["بنات"] = daughters_share * estate_for_inheritance

    # توزيع العصبة
    if remainder > 0 and total_asaba_units > 0:
        unit_value = remainder * estate_for_inheritance / total_asaba_units
        for key, units in asaba.items():
            shares[key] = shares.get(key, 0) + units * unit_value

    # الطباعة التفصيلية
    print("\nتوزيع الميراث بعد خصم الديون والوصايا:")
    for heir, amount in shares.items():
        print(f"{heir}: {amount:.2f} ({amount/estate:.3%} من التركة الأصلية)")

    return shares


# مصفوفة بيانات متنوعة للاختبار
test_cases = [
    {
        "description": "حالة 1: زوج، أب، أم، 2 أبناء ذكور، 1 بنت، ديون 10000، وصايا 15000",
        "estate": 100000,
        "debts": 10000,
        "wills": 15000,
        "heirs": {
            "زوج": True,
            "زوجات": 0,
            "أب": True,
            "أم": True,
            "أبناء_ذكور": 2,
            "بنات": 1,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 2: زوجة، أم، 3 بنات، لا أبناء ذكور، ديون 5000، وصايا 10000",
        "estate": 120000,
        "debts": 5000,
        "wills": 10000,
        "heirs": {
            "زوج": False,
            "زوجات": 1,
            "أب": False,
            "أم": True,
            "أبناء_ذكور": 0,
            "بنات": 3,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 3: أب، أم، بنت واحدة، لا زوج ولا أبناء ذكور، ديون 0، وصايا 0",
        "estate": 80000,
        "debts": 0,
        "wills": 0,
        "heirs": {
            "زوج": False,
            "زوجات": 0,
            "أب": True,
            "أم": True,
            "أبناء_ذكور": 0,
            "بنات": 1,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 4: زوج، لا أب، لا أم، 1 ابن ذكر، 2 بنات، ديون 2000، وصايا 0",
        "estate": 50000,
        "debts": 2000,
        "wills": 0,
        "heirs": {
            "زوج": True,
            "زوجات": 0,
            "أب": False,
            "أم": False,
            "أبناء_ذكور": 1,
            "بنات": 2,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 5: أم فقط، لا أب، لا أبناء، لا زوج، ديون 0، وصايا 5000 (تجاوز الثلث)",
        "estate": 30000,
        "debts": 0,
        "wills": 5000,
        "heirs": {
            "زوج": False,
            "زوجات": 0,
            "أب": False,
            "أم": True,
            "أبناء_ذكور": 0,
            "بنات": 0,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    }
]

# تنفيذ الاختبارات
for case in test_cases:
    print("\n" + "="*60)
    print(case["description"])
    calculate_inheritance(case["estate"], case["heirs"], debts=case["debts"], wills=case["wills"])

In [17]:

def calculate_inheritance(estate, heirs, debts=0, wills=0):
    """
    توزيع الميراث مع خصم الديون والوصايا.
    estate: قيمة التركة الكلية قبل الخصم
    heirs: بيانات الورثة وعددهم (قاموس)
    debts: مجموع الديون المستحقة على المتوفى (رقم)
    wills: قيمة الوصايا (يجب ألا تتجاوز ثلث التركة بعد خصم الديون)
    """

    print(f"التركة الكلية قبل الخصم: {estate}")
    print(f"الديون المستحقة: {debts}")
    print(f"الوصايا (حتى ثلث التركة المتبقية): {wills}")

    if debts > estate:
        print("الديون أكبر من التركة، لا ميراث للورثة.")
        return {}
    estate_after_debts = estate - debts

    max_will = estate_after_debts / 3
    if wills > max_will:
        print(f"الوصايا تتجاوز الثلث المسموح به ({max_will:.2f}). سيتم تقليلها إلى الثلث.")
        wills = max_will
    estate_for_inheritance = estate_after_debts - wills

    print(f"التركة المتبقية للتوزيع بعد خصم الديون والوصايا: {estate_for_inheritance}")

    # 1. نصيب الزوج/الزوجة
    husband_share = 0
    wife_share = 0
    if heirs.get("زوج", False):
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0:
            husband_share = 1/4
        else:
            husband_share = 1/2

    if heirs.get("زوجات", 0) > 0:
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0:
            wife_share = 1/8
        else:
            wife_share = 1/4

    # 2. نصيب الأم
    mother_share = 0
    if heirs.get("أم", False):
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0 or heirs.get("إخوة_ذكور", 0) + heirs.get("إخوة_إناث", 0) > 0:
            mother_share = 1/6
        else:
            mother_share = 1/3

    # 3. نصيب الأب
    father_share = 0
    father_is_asaba = False
    if heirs.get("أب", False):
        if heirs.get("أبناء_ذكور", 0) + heirs.get("بنات", 0) > 0 or heirs.get("إخوة_ذكور", 0) > 0:
            father_share = 1/6
        else:
            father_share = 1/6
            father_is_asaba = True

    # 4. نصيب البنات
    daughters_share = 0
    daughters_count = heirs.get("بنات", 0)
    sons_count = heirs.get("أبناء_ذكور", 0)
    if daughters_count > 0:
        if sons_count == 0:
            if daughters_count == 1:
                daughters_share = 1/2
            elif daughters_count >= 2:
                daughters_share = 2/3
        else:
            daughters_share = 0  # يرثن مع الأولاد حسب العصبة

    # 5. مجموع الفروض
    total_fixed_shares = husband_share + wife_share + mother_share + father_share + daughters_share

    # 6. الرد إذا تجاوزت الفروض التركة
    if total_fixed_shares > 1:
        factor = 1 / total_fixed_shares
        husband_share *= factor
        wife_share *= factor
        mother_share *= factor
        father_share *= factor
        daughters_share *= factor
        total_fixed_shares = 1

    # 7. الباقي للعصبة
    remainder = 1 - total_fixed_shares

    asaba = {}
    total_asaba_units = 0

    if sons_count > 0:
        asaba["أبناء_ذكور"] = sons_count * 2
        total_asaba_units += sons_count * 2
        if daughters_count > 0:
            asaba["بنات"] = daughters_count * 1
            total_asaba_units += daughters_count * 1
    else:
        if father_is_asaba:
            asaba["أب"] = 1
            total_asaba_units += 1

    shares = {}

    # توزيع الفروض
    if husband_share > 0:
        shares["زوج"] = husband_share * estate_for_inheritance
    if wife_share > 0:
        shares["زوجة"] = wife_share * estate_for_inheritance
    if mother_share > 0:
        shares["أم"] = mother_share * estate_for_inheritance
    if father_share > 0:
        shares["أب"] = father_share * estate_for_inheritance
    if daughters_share > 0 and sons_count == 0:
        shares["بنات"] = daughters_share * estate_for_inheritance

    # توزيع العصبة
    if remainder > 0 and total_asaba_units > 0:
        unit_value = remainder * estate_for_inheritance / total_asaba_units
        for key, units in asaba.items():
            shares[key] = shares.get(key, 0) + units * unit_value

    # طباعة النتائج
    print("\nتوزيع الميراث بعد خصم الديون والوصايا:")
    if not shares:
        print("لا يوجد ورثة أو لا ميراث للتوزيع.")
    else:
        for heir, amount in shares.items():
            print(f"{heir}: {amount:.2f} ({amount/estate:.3%} من التركة الأصلية)")

    return shares


# عينات للاختبار
test_cases = [
    {
        "description": "حالة 1: زوج، أب، أم، 2 أبناء ذكور، 1 بنت، ديون 10000، وصايا 15000",
        "estate": 100000,
        "debts": 10000,
        "wills": 15000,
        "heirs": {
            "زوج": True,
            "زوجات": 0,
            "أب": True,
            "أم": True,
            "أبناء_ذكور": 2,
            "بنات": 1,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 2: زوجة، أم، 3 بنات، لا أبناء ذكور، ديون 5000، وصايا 10000",
        "estate": 120000,
        "debts": 5000,
        "wills": 10000,
        "heirs": {
            "زوج": False,
            "زوجات": 1,
            "أب": False,
            "أم": True,
            "أبناء_ذكور": 0,
            "بنات": 3,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 3: أب، أم، بنت واحدة، لا زوج ولا أبناء ذكور، ديون 0، وصايا 0",
        "estate": 80000,
        "debts": 0,
        "wills": 0,
        "heirs": {
            "زوج": False,
            "زوجات": 0,
            "أب": True,
            "أم": True,
            "أبناء_ذكور": 0,
            "بنات": 1,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 4: زوج، لا أب، لا أم، 1 ابن ذكر، 2 بنات، ديون 2000، وصايا 0",
        "estate": 50000,
        "debts": 2000,
        "wills": 0,
        "heirs": {
            "زوج": True,
            "زوجات": 0,
            "أب": False,
            "أم": False,
            "أبناء_ذكور": 1,
            "بنات": 2,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    },
    {
        "description": "حالة 5: أم فقط، لا أب، لا أبناء، لا زوج، ديون 0، وصايا 5000 (تجاوز الثلث)",
        "estate": 30000,
        "debts": 0,
        "wills": 5000,
        "heirs": {
            "زوج": False,
            "زوجات": 0,
            "أب": False,
            "أم": True,
            "أبناء_ذكور": 0,
            "بنات": 0,
            "إخوة_ذكور": 0,
            "إخوة_إناث": 0,
            "جد": False,
            "جدة": False
        }
    }
]

# تنفيذ الاختبارات
for case in test_cases:
    print("\n" + "="*60)
    print(case["description"])
    calculate_inheritance(case["estate"], case["heirs"], debts=case["debts"], wills=case["wills"])


حالة 1: زوج، أب، أم، 2 أبناء ذكور، 1 بنت، ديون 10000، وصايا 15000
التركة الكلية قبل الخصم: 100000
الديون المستحقة: 10000
الوصايا (حتى ثلث التركة المتبقية): 15000
التركة المتبقية للتوزيع بعد خصم الديون والوصايا: 75000

توزيع الميراث بعد خصم الديون والوصايا:
زوج: 18750.00 (18.750% من التركة الأصلية)
أم: 12500.00 (12.500% من التركة الأصلية)
أب: 12500.00 (12.500% من التركة الأصلية)
أبناء_ذكور: 25000.00 (25.000% من التركة الأصلية)
بنات: 6250.00 (6.250% من التركة الأصلية)

حالة 2: زوجة، أم، 3 بنات، لا أبناء ذكور، ديون 5000، وصايا 10000
التركة الكلية قبل الخصم: 120000
الديون المستحقة: 5000
الوصايا (حتى ثلث التركة المتبقية): 10000
التركة المتبقية للتوزيع بعد خصم الديون والوصايا: 105000

توزيع الميراث بعد خصم الديون والوصايا:
زوجة: 13125.00 (10.938% من التركة الأصلية)
أم: 17500.00 (14.583% من التركة الأصلية)
بنات: 70000.00 (58.333% من التركة الأصلية)

حالة 3: أب، أم، بنت واحدة، لا زوج ولا أبناء ذكور، ديون 0، وصايا 0
التركة الكلية قبل الخصم: 80000
الديون المستحقة: 0
الوصايا (حتى ثلث التركة المت

In [ ]:

import random

def random_heirs():
    """
    توليد بيانات ورثة عشوائية مع احتمالات وجود كل وارث وعددهم.
    """
    heirs = {
        "زوج": random.choice([True, False]),
        "زوجات": 0,  # نفترض زوجة واحدة كحد أقصى
        "أب": random.choice([True, False]),
        "أم": random.choice([True, False]),
        "أبناء_ذكور": random.randint(0, 3),
        "بنات": random.randint(0, 3),
        "إخوة_ذكور": random.randint(0, 2),
        "إخوة_إناث": random.randint(0, 2),
        "جد": random.choice([True, False]),
        "جدة": random.choice([True, False])
    }
    # لا نجعل الزوج والزوجة معًا في نفس الوقت (لأنهما لا يجتمعان)
    if heirs["زوج"] and heirs["زوجات"] > 0:
        heirs["زوجات"] = 0
    # احتمال وجود زوجة فقط (مثلاً 30%)
    if not heirs["زوج"]:
        heirs["زوجات"] = random.choices([0,1], weights=[0.7,0.3])[0]
    else:
        heirs["زوجات"] = 0
    return heirs

def generate_random_test_cases(n=5):
    """
    توليد n حالة اختبار عشوائية مع توزيع ميراث لكل حالة.
    """
    for i in range(n):
        estate = random.randint(50000, 200000)  # قيمة التركة بين 50 ألف و 200 ألف
        debts = random.randint(0, int(estate * 0.3))  # ديون حتى 30% من التركة
        wills_max = (estate - debts) / 3
        wills = round(random.uniform(0, wills_max), 2)  # وصايا حتى ثلث التركة المتبقية

        heirs = random_heirs()

        print("\n" + "="*60)
        print(f"حالة اختبار عشوائية #{i+1}")
        print(f"قيمة التركة: {estate}, الديون: {debts}, الوصايا: {wills}")
        print(f"الورثة وعددهم: {heirs}")

        calculate_inheritance(estate, heirs, debts, wills)

# استدعاء التوليد والتنفيذ
generate_random_test_cases(5)


حالة اختبار عشوائية #1
قيمة التركة: 155389, الديون: 4569, الوصايا: 45363.73
الورثة وعددهم: {'زوج': True, 'زوجات': 0, 'أب': True, 'أم': False, 'أبناء_ذكور': 2, 'بنات': 2, 'إخوة_ذكور': 2, 'إخوة_إناث': 1, 'جد': False, 'جدة': False}
التركة الكلية قبل الخصم: 155389
الديون المستحقة: 4569
الوصايا (حتى ثلث التركة المتبقية): 45363.73
التركة المتبقية للتوزيع بعد خصم الديون والوصايا: 105456.26999999999

توزيع الميراث بعد خصم الديون والوصايا:
زوج: 26364.07 (16.966% من التركة الأصلية)
أب: 17576.04 (11.311% من التركة الأصلية)
أبناء_ذكور: 41010.77 (26.392% من التركة الأصلية)
بنات: 20505.39 (13.196% من التركة الأصلية)

حالة اختبار عشوائية #2
قيمة التركة: 159797, الديون: 440, الوصايا: 33262.25
الورثة وعددهم: {'زوج': True, 'زوجات': 0, 'أب': True, 'أم': True, 'أبناء_ذكور': 2, 'بنات': 2, 'إخوة_ذكور': 1, 'إخوة_إناث': 0, 'جد': True, 'جدة': True}
التركة الكلية قبل الخصم: 159797
الديون المستحقة: 440
الوصايا (حتى ثلث التركة المتبقية): 33262.25
التركة المتبقية للتوزيع بعد خصم الديون والوصايا: 126094.75

توزيع 

In [13]:
import random
import csv

def calculate_inheritance(estate, heirs, debts=0, wills=0):
    """
    Calculate inheritance shares after deducting debts and wills.
    estate: total estate value available for distribution (after debts and wills)
    heirs: dictionary of heirs and their counts/presence
    debts: total debts (not used in calculation here, assumed already deducted)
    wills: total wills (not used in calculation here, assumed already deducted)
    """

    husband_share = 0
    wife_share = 0
    if heirs.get("husband", False):
        if heirs.get("sons", 0) + heirs.get("daughters", 0) > 0:
            husband_share = 1/4
        else:
            husband_share = 1/2

    if heirs.get("wives", 0) > 0:
        if heirs.get("sons", 0) + heirs.get("daughters", 0) > 0:
            wife_share = 1/8
        else:
            wife_share = 1/4

    mother_share = 0
    if heirs.get("mother", False):
        if heirs.get("sons", 0) + heirs.get("daughters", 0) > 0 or \
           heirs.get("brothers_m", 0) + heirs.get("sisters_m", 0) > 0:
            mother_share = 1/6
        else:
            mother_share = 1/3

    father_share = 0
    father_is_asaba = False
    if heirs.get("father", False):
        if heirs.get("sons", 0) + heirs.get("daughters", 0) > 0 or heirs.get("brothers_m", 0) > 0:
            father_share = 1/6
        else:
            father_share = 1/6
            father_is_asaba = True

    daughters_share = 0
    daughters_count = heirs.get("daughters", 0)
    sons_count = heirs.get("sons", 0)
    if daughters_count > 0:
        if sons_count == 0:
            if daughters_count == 1:
                daughters_share = 1/2
            elif daughters_count >= 2:
                daughters_share = 2/3
        else:
            daughters_share = 0  # daughters inherit with sons as residuaries

    total_fixed_shares = husband_share + wife_share + mother_share + father_share + daughters_share

    if total_fixed_shares > 1:
        factor = 1 / total_fixed_shares
        husband_share *= factor
        wife_share *= factor
        mother_share *= factor
        father_share *= factor
        daughters_share *= factor
        total_fixed_shares = 1

    remainder = 1 - total_fixed_shares

    residuaries = {}
    total_residuary_units = 0

    if sons_count > 0:
        residuaries["sons"] = sons_count * 2
        total_residuary_units += sons_count * 2
        if daughters_count > 0:
            residuaries["daughters"] = daughters_count * 1
            total_residuary_units += daughters_count * 1
    else:
        if father_is_asaba:
            residuaries["father"] = 1
            total_residuary_units += 1

    shares = {}

    if husband_share > 0:
        shares["husband"] = husband_share * estate
    if wife_share > 0:
        shares["wife"] = wife_share * estate
    if mother_share > 0:
        shares["mother"] = mother_share * estate
    if father_share > 0:
        shares["father"] = father_share * estate
    if daughters_share > 0 and sons_count == 0:
        shares["daughters"] = daughters_share * estate

    if remainder > 0 and total_residuary_units > 0:
        unit_value = remainder * estate / total_residuary_units
        for key, units in residuaries.items():
            shares[key] = shares.get(key, 0) + units * unit_value

    return shares

def random_heirs():
    """
    Generate random heirs data with reasonable probabilities.
    """
    heirs = {
        "husband": random.choice([True, False]),
        "wives": 0,
        "father": random.choice([True, False]),
        "mother": random.choice([True, False]),
        "sons": random.randint(0, 3),
        "daughters": random.randint(0, 3),
        "brothers_m": random.randint(0, 2),
        "sisters_m": random.randint(0, 2),
        "grandfather": random.choice([True, False]),
        "grandmother": random.choice([True, False])
    }
    # Adjust spouse presence (husband and wives exclusive)
    if heirs["husband"]:
        heirs["wives"] = 0
    else:
        heirs["wives"] = random.choices([0,1], weights=[0.7,0.3])[0]
    return heirs

def generate_dataset(num_records=20000, filename='inheritance_dataset.csv'):
    fieldnames = [
        'record_id', 'estate', 'debts', 'wills',
        'husband', 'wives', 'father', 'mother', 'sons', 'daughters',
        'brothers_m', 'sisters_m', 'grandfather', 'grandmother',
        'share_husband', 'share_wife', 'share_father', 'share_mother',
        'share_sons', 'share_daughters'
    ]

    with open(filename, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for i in range(1, num_records + 1):
            estate = random.randint(50000, 200000)
            debts = random.randint(0, int(estate * 0.3))
            max_will = (estate - debts) / 3
            wills = round(random.uniform(0, max_will), 2)

            heirs = random_heirs()

            estate_for_distribution = estate - debts - wills
            shares = calculate_inheritance(estate_for_distribution, heirs)

            row = {
                'record_id': i,
                'estate': estate,
                'debts': debts,
                'wills': wills,
                'husband': heirs['husband'],
                'wives': heirs['wives'],
                'father': heirs['father'],
                'mother': heirs['mother'],
                'sons': heirs['sons'],
                'daughters': heirs['daughters'],
                'brothers_m': heirs['brothers_m'],
                'sisters_m': heirs['sisters_m'],
                'grandfather': heirs['grandfather'],
                'grandmother': heirs['grandmother'],
                'share_husband': round(shares.get('husband', 0), 2),
                'share_wife': round(shares.get('wife', 0), 2),
                'share_father': round(shares.get('father', 0), 2),
                'share_mother': round(shares.get('mother', 0), 2),
                'share_sons': round(shares.get('sons', 0), 2),
                'share_daughters': round(shares.get('daughters', 0), 2),
            }

            writer.writerow(row)

            if i % 1000 == 0:
                print(f"{i} records generated...")

    print(f"\nDataset generation complete. File saved as '{filename}'.")

# To generate the dataset, just call:
# generate_dataset()


In [15]:
# Paste here your full code including calculate_inheritance and generate_dataset functions

generate_dataset()

from google.colab import files
files.download('inheritance_dataset.csv')


1000 records generated...
2000 records generated...
3000 records generated...
4000 records generated...
5000 records generated...
6000 records generated...
7000 records generated...
8000 records generated...
9000 records generated...
10000 records generated...
11000 records generated...
12000 records generated...
13000 records generated...
14000 records generated...
15000 records generated...
16000 records generated...
17000 records generated...
18000 records generated...
19000 records generated...
20000 records generated...

Dataset generation complete. File saved as 'inheritance_dataset.csv'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from google.colab import files
files.download('inheritance_dataset.csv')


FileNotFoundError: Cannot find file: inheritance_dataset.csv